In [14]:
import os
os.chdir('G:\\bag of words')
import nltk

In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [7]:
# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [11]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords


In [12]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [15]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [18]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [19]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\sai ram\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably op

Parsing sentences from unlabeled set


C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\sai ram\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sa

In [20]:
print(len(sentences))

795538


In [21]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [22]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [23]:
import logging

In [24]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [25]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [27]:
from gensim.models import word2vec

C:\Users\sai ram\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-12-20 22:42:21,940 : INFO : 'pattern' package not found; tag filters are not available for English


In [28]:
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


2018-12-20 22:43:22,292 : INFO : collecting all words and their counts
2018-12-20 22:43:22,307 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-20 22:43:22,364 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-12-20 22:43:22,414 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-12-20 22:43:22,446 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-12-20 22:43:22,514 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-12-20 22:43:22,565 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-12-20 22:43:22,615 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-12-20 22:43:22,665 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-12-20 22:43:22,712 : INFO : PROGRESS: 

2018-12-20 22:43:26,298 : INFO : PROGRESS: at sentence #720000, processed 16105489 words, keeping 118221 word types
2018-12-20 22:43:26,366 : INFO : PROGRESS: at sentence #730000, processed 16331870 words, keeping 118954 word types
2018-12-20 22:43:26,416 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2018-12-20 22:43:26,463 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-12-20 22:43:26,516 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-12-20 22:43:26,563 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-12-20 22:43:26,640 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-12-20 22:43:26,697 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-12-20 22:43:26,715 : INFO : collected 123504 word types from a corp

2018-12-20 22:44:16,526 : INFO : EPOCH 3 - PROGRESS: at 67.29% examples, 655330 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:17,542 : INFO : EPOCH 3 - PROGRESS: at 72.44% examples, 654972 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:18,543 : INFO : EPOCH 3 - PROGRESS: at 77.66% examples, 655468 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:19,558 : INFO : EPOCH 3 - PROGRESS: at 82.94% examples, 656267 words/s, in_qsize 8, out_qsize 0
2018-12-20 22:44:20,560 : INFO : EPOCH 3 - PROGRESS: at 88.13% examples, 656403 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:21,577 : INFO : EPOCH 3 - PROGRESS: at 93.46% examples, 657128 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:22,578 : INFO : EPOCH 3 - PROGRESS: at 98.17% examples, 653962 words/s, in_qsize 7, out_qsize 0
2018-12-20 22:44:22,923 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-20 22:44:22,942 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-20 22:44:22,950 : I

In [29]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)


2018-12-20 22:46:15,004 : INFO : precomputing L2-norms of word weight vectors


In [30]:
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-12-20 22:46:46,088 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-12-20 22:46:46,090 : INFO : not storing attribute vectors_norm
2018-12-20 22:46:46,116 : INFO : not storing attribute cum_table
2018-12-20 22:46:47,484 : INFO : saved 300features_40minwords_10context


In [31]:
 model.doesnt_match("man woman child kitchen".split())

C:\Users\sai ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
C:\Users\sai ram\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [32]:
model.most_similar("man")

C:\Users\sai ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\sai ram\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6334397792816162),
 ('lady', 0.5824787020683289),
 ('lad', 0.579102098941803),
 ('monk', 0.5246774554252625),
 ('men', 0.5227718949317932),
 ('person', 0.5140311121940613),
 ('guy', 0.5097619295120239),
 ('chap', 0.5096339583396912),
 ('millionaire', 0.5076737403869629),
 ('farmer', 0.5016855001449585)]

In [33]:
model.most_similar("monkey")

C:\Users\sai ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\sai ram\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('chicken', 0.6779242157936096),
 ('turtle', 0.6682215332984924),
 ('robot', 0.657378077507019),
 ('rabbit', 0.6402344703674316),
 ('dinosaur', 0.6398425102233887),
 ('rat', 0.6384816765785217),
 ('giant', 0.6384641528129578),
 ('gorilla', 0.6205796599388123),
 ('soup', 0.604804277420044),
 ('bowl', 0.6009451150894165)]

In [34]:
model.most_similar("awful")

C:\Users\sai ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\sai ram\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.782239556312561),
 ('horrible', 0.7387335896492004),
 ('dreadful', 0.7276715636253357),
 ('horrid', 0.7180150747299194),
 ('atrocious', 0.7161433696746826),
 ('abysmal', 0.6995251178741455),
 ('horrendous', 0.692312479019165),
 ('appalling', 0.657764196395874),
 ('lousy', 0.6431196928024292),
 ('amateurish', 0.617017924785614)]

In [36]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2018-12-20 23:03:18,180 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-12-20 23:03:19,126 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-12-20 23:03:19,127 : INFO : setting ignored attribute vectors_norm to None
2018-12-20 23:03:19,128 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-12-20 23:03:19,131 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-12-20 23:03:19,134 : INFO : setting ignored attribute cum_table to None
2018-12-20 23:03:19,136 : INFO : loaded 300features_40minwords_10context


In [39]:
model["flower"]

C:\Users\sai ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.09055354,  0.1463171 , -0.06008542, -0.02154416, -0.00859701,
       -0.00713478, -0.0015337 ,  0.02209278,  0.0482654 , -0.02417834,
        0.01453944,  0.09131053, -0.01200016,  0.05707933,  0.0088407 ,
        0.05334655,  0.13365936, -0.00328466, -0.02480497, -0.10646852,
        0.03922846,  0.00381758,  0.0260873 ,  0.22292404, -0.03467258,
        0.07119861, -0.08352372, -0.07849441,  0.03079041,  0.01104717,
       -0.01417322, -0.04866726,  0.0359098 ,  0.06147364, -0.02884374,
        0.01493174,  0.04310409,  0.07644881, -0.00189303, -0.07377511,
        0.00888221,  0.05229086, -0.03775471, -0.04197915, -0.06843879,
        0.06017553,  0.04128376,  0.01455795, -0.04861053, -0.00917148,
       -0.02096473,  0.05581229, -0.06876916,  0.03942876, -0.14354055,
        0.06027647,  0.03236742, -0.00854953,  0.08471157,  0.10710889,
       -0.07508965, -0.07156443, -0.01123626,  0.05661638,  0.03756279,
        0.05227413, -0.04571367,  0.015947  ,  0.02598168,  0.00

In [40]:
import numpy as np

In [41]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec